In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = "../input/utkfacenew/UTKFace/"
files = os.listdir(path)
size = len(files)
print("Total samples:",size)
print(files[0])

In [ ]:
import cv2
ages=[]
genders=[]
images=[]
for fle in files:
  age=int(fle.split('_')[0])
  gender=int(fle.split('_')[1])
  total=path+fle
  print(total)
  image=cv2.imread(total)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image= cv2.resize(image,(48,48))
  images.append(image)
  ages.append(age)
  genders.append(gender)

In [ ]:
labels=[]
i=0
while i<len(ages):
  label=[]
  label.append([ages[i]])
  label.append([genders[i]])
  labels.append(label)
  i+=1

In [ ]:
from sklearn.model_selection import train_test_split

images_f=np.array(images)
labels_f=np.array(labels)
images_f_2=images_f/255
X_train, X_test, Y_train, Y_test= train_test_split(images_f_2, labels_f,test_size=0.20)

In [ ]:
Y_train[0:5]

In [ ]:
Y_train_2=[Y_train[:,1],Y_train[:,0]]
Y_test_2=[Y_test[:,1],Y_test[:,0]]

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf


inputs = Input(shape=(48,48,3))
  
conv_1= Conv2D(32, strides=(1, 1), kernel_size=(3, 3),activation='relu')(inputs)
maxp_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)
conv_2 = Conv2D(64, strides=(1, 1), kernel_size=(3, 3),activation='relu')(maxp_1)
maxp_2 = MaxPooling2D(pool_size = (2, 2)) (conv_2)
conv_3 = Conv2D(128, strides=(1, 1), kernel_size=(3, 3),activation='relu')(maxp_2)
maxp_3 = MaxPooling2D(pool_size = (2, 2)) (conv_3)
flatten= Flatten() (maxp_3)
dense_1= Dense(64,activation='relu')(flatten)
dense_2= Dense(64,activation='relu')(flatten)
drop_1=Dropout(0.2)(dense_1)
drop_2=Dropout(0.4)(dense_2)
output_1= Dense(1,activation="sigmoid",name='sex_out')(drop_1)
output_2= Dense(1,name='age_out')(drop_2)
model = Model(inputs=[inputs], outputs=[output_1,output_2])
opt = Adam(learning_rate=0.001)
model.compile(loss=["binary_crossentropy", "rmse"], optimizer= opt, metrics=["accuracy","rmse"])

model.summary()

history1 = model.fit(X_train,Y_train_2,validation_data=(X_test,Y_test_2), epochs=40, batch_size = 8)

# training  0.75 , 3 layers, with dropout for age as 0.1 (decent age results)

In [ ]:
history1 = model.fit(X_train,Y_train_2,validation_data=(X_test,Y_test_2), epochs=30, batch_size = 16)

In [ ]:
import matplotlib.pyplot as plt

ax1 = plt.plot(history1.history['sex_out_accuracy'])
ax2 = plt.plot(history1.history['val_sex_out_accuracy'])
plt.title('gender model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='center right')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history1.history['age_out_mse'])
plt.plot(history1.history['val_age_out_mse'])
plt.title('Age model MSE')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='center right')
plt.show()

In [ ]:
def display(img):
    plt.imshow(img[:,:,0])
    plt.set_cmap('gray')
    plt.show()

    
def get_gender(prob):
    if prob < 0.5:return "Male"
    else: return "Female"

def get_result(sample):
    sample = sample/255
    val = model.predict( np.array([ sample ]) )    
    age = val[1]
    gender = get_gender(val[0])
    #print("Values:",val,"\nPredicted Gender:",gender,"Predicted Age:",age)
    print("Predicted Gender:",gender,"Predicted Age:",age)
    
indexes = [500,59,80,2,4546,7,9,256,45]
for idx in indexes:
    sample = images[idx]
    display(sample)
    print("Actual Gender:",get_gender(genders[idx]),"Age:",ages[idx])
    res = get_result(sample)

In [ ]:
path = "../input/ghoshpred1/"
files = os.listdir(path)
size = len(files)
print("Total samples:",size)
print(files[0])

In [ ]:
import cv2
ages_1=[]
genders_1=[]
images_1=[]
for fle in files:
  age_1=int(fle.split('_')[0])
  gender_1=int(fle.split('_')[1])
  total_1=path+fle
  print(total_1)
  image_1=cv2.imread(total_1)
  image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
  image_1= cv2.resize(image_1,(48,48))
  images_1.append(image_1)
  ages_1.append(age_1)
  genders_1.append(gender_1)

In [ ]:
images_2=np.array(images_1)
images_pred=images_2/255

pred = model.predict( np.array(images_pred) )
display(images_pred[0])
age_pred = pred[1]
gender_pred = get_gender(pred[0])
print("Predicted Gender:",gender_pred,"\nPredicted Age:",age_pred)

In [ ]:
path = "../input/ghoshpred2/"
files = os.listdir(path)
size = len(files)
print("Total samples:",size)
print(files[0])

In [ ]:
import cv2
ages_1=[]
genders_1=[]
images_1=[]
for fle in files:
  age_1=int(fle.split('_')[0])
  gender_1=int(fle.split('_')[1])
  total_1=path+fle
  print(total_1)
  image_1=cv2.imread(total_1)
  image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2RGB)
  image_1= cv2.resize(image_1,(48,48))
  images_1.append(image_1)
  ages_1.append(age_1)
  genders_1.append(gender_1)

In [ ]:
images_2=np.array(images_1)
images_pred=images_2/255

pred = model.predict( np.array(images_pred) )
display(images_pred[0])
age_pred = pred[1]
gender_pred = get_gender(pred[0])
print("Predicted Gender:",gender_pred,"\nPredicted Age:",age_pred)